In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import os
from sklearn.metrics import confusion_matrix, roc_auc_score

In [12]:
os.chdir("/home/ubuntu/extraVol/ARVAR/iSNVs")
df = pd.read_csv('IntraSnv_results/ampseq_ConsTest_freq_1_0.csv')
#df = pd.read_csv('IntraSnv_results/metaseq_ConsTest_freq_1_0.csv')
mask = df[['Var_Al_RelPos', 'Ref_Al_RelPos']].isna().any(axis=1)
varNa = df[mask]
dfFilt = df.dropna(subset=['Var_Al_RelPos', 'Ref_Al_RelPos'], how='any').reset_index(drop = True)

In [13]:
colOpt1 = ['ALLELE.FREQUENCY', 'STRAND.BIAS', 'QUAL', 'Var_Al_RelPos', 'Ref_Al_RelPos',  'meandepth', 'meanbaseq']
colOpt5 = ['ALLELE.FREQUENCY', 'STRAND.BIAS', 'QUAL', 'Var_Al_RelPos', 'Ref_Al_RelPos',  'meandepth', 'meanbaseq', 'Sample', 'Sample_AlignPos_Ref_Var']
X = dfFilt[colOpt5]
y = dfFilt['ConsTest']
varNa = X[X.isna().any(axis=1)]
respNa = y[y.isna()]
X_train_1, X_test_1, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X_train_1[colOpt1]
X_test = X_test_1[colOpt1]

In [14]:
# Create and fit the Random Forest model on the balanced training dataset
model = RandomForestClassifier(
    criterion='gini',
    n_estimators=3000,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=None,
    oob_score=True,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

RandomForestClassifier(max_features=None, n_estimators=3000, n_jobs=-1,
                       oob_score=True, random_state=42)

In [15]:
# Predict probabilities on the test set
y_pred_proba = model.predict_proba(X_test)[:, 1]
# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC Score: {auc_score}")
print("%.4f" % model.oob_score_)



y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
# Extract the true negative, false positive, false negative, and true positive values
tn, fp, fn, tp = cm.ravel()

# Calculate class error rates
class_0_error = fp / (tn + fp)  # Class 0 (ConsTest = 0)
class_1_error = fn / (fn + tp)  # Class 1 (ConsTest = 1)

# Print the class error rates
print(f"Class 0 Error Rate: {class_0_error:.4f}")
print(f"Class 1 Error Rate: {class_1_error:.4f}")

AUC Score: 0.9082160684698173
0.9238
Class 0 Error Rate: 0.0129
Class 1 Error Rate: 0.4390


In [16]:
def SumPredict(df, minFreq, maxFreq):
  corPredict = []
  wrongPredict = []
  dfFilt = df[(df["ALLELE.FREQUENCY"] >= minFreq) & (df["ALLELE.FREQUENCY"] <= maxFreq)].reset_index().drop(["index"], axis =1)
  for i in range(len(dfFilt.index)):
    if dfFilt.loc[i, "Predict_val"] == dfFilt.loc[i, "ConsTest"]:
      corPredict.append(dfFilt.loc[i, "Sample_AlignPos_Ref_Var"])
    else:
      wrongPredict.append(dfFilt.loc[i, "Sample_AlignPos_Ref_Var"])
  numbCorPred = len(corPredict)
  numbWrongPred = len(wrongPredict)
  return numbCorPred, numbWrongPred

def SumPredictPerSamp(df, minFreq, maxFreq):
  snvs = []
  truePos = []
  falsePos = []
  Samples = []
  totalPos = []
  corIdent = []
  wrongIdnet = []
  falseNeg = []
  dfFilt = df[(df["ALLELE.FREQUENCY"] >= minFreq) & (df["ALLELE.FREQUENCY"] <= maxFreq)].reset_index().drop(["index"], axis =1)
  samples = list(pd.unique(df["Sample"]))
  for sample in samples:
    corPredict = []
    wrongPredict = []
    curTruePos = []
    curFalsePos = []
    curFalseNeg = []
    dfSub = dfFilt[dfFilt["Sample"] == sample].reset_index().drop(["index"], axis =1)
    for i in range(len(dfSub.index)):
      if dfSub.loc[i, "Predict_val"] == dfSub.loc[i, "ConsTest"]:
        corPredict.append(dfSub.loc[i, "Sample_AlignPos_Ref_Var"])
      else:
        wrongPredict.append(dfSub.loc[i, "Sample_AlignPos_Ref_Var"])
        if dfSub.loc[i, "Predict_val"] == 1:
          curFalsePos.append(dfSub.loc[i, "Sample_AlignPos_Ref_Var"])
        elif dfSub.loc[i, "Predict_val"] == 0:
          curFalseNeg.append(dfSub.loc[i, "Sample_AlignPos_Ref_Var"])
    numbCorPred = len(corPredict)
    numbWrongPred = len(wrongPredict)
    dfPos = dfSub[dfSub["ConsTest"] ==1].reset_index().drop(["index"], axis =1)
    curPos = dfPos['Sample_AlignPos_Ref_Var'].nunique()
    curSnvs = dfSub['Sample_AlignPos_Ref_Var'].nunique()
    for i in range(len(dfPos.index)):
      if dfPos.loc[i, "Predict_val"] == dfPos.loc[i, "ConsTest"]:
        curTruePos.append(dfPos.loc[i, "Sample_AlignPos_Ref_Var"])
        
    numbTruePos = len(curTruePos)
    numbFalsePos = len(curFalsePos)
    numbFalseNeg = len(curFalseNeg)
    
    #append values
    snvs.append(curSnvs)
    corIdent.append(numbCorPred)
    wrongIdnet.append(numbWrongPred)
    Samples.append(sample)
    totalPos.append(curPos)
    truePos.append(numbTruePos)
    falsePos.append(numbFalsePos)
    falseNeg.append(numbFalseNeg)
  combDat = pd.DataFrame({"Sample": Samples, "Total_SNVs": snvs, "Correctly_identified": corIdent, "Incorrectly_identified": wrongIdnet , 'False_Positive': falsePos, "False_Negative":falseNeg, 
                          "Total_Positive_Truth": totalPos, "True_Positive_Ident": truePos})
  return combDat

In [17]:
# make data with Ids and survival
X_test_1["Predict_val"] = y_pred
X_test_1['ConsTest'] = y_test

numbCorPred, numbWrongPred = SumPredict(df = X_test_1, minFreq = 0, maxFreq = 1)
print(numbCorPred / (numbCorPred + numbWrongPred) * 100)

sumTest = SumPredictPerSamp(df = X_test_1, minFreq = 0, maxFreq = 1)

print(sumTest["True_Positive_Ident"].sum() / sumTest["Total_Positive_Truth"].sum() * 100)

print(sumTest['Correctly_identified'].sum() / sumTest['Total_SNVs'].sum() * 100)

print(f"AUC Score: {auc_score}")


92.71913465124953
56.104033970276014
92.71913465124953
AUC Score: 0.9082160684698173


In [18]:
print(sumTest["True_Positive_Ident"].sum())
print(sumTest["Total_Positive_Truth"].sum())
print(sumTest["False_Positive"].sum())
print(sumTest["False_Negative"].sum())

1057
1884
149
827


In [19]:
from sklearn.metrics import roc_curve, auc
import numpy as np
# Calculate the AUC score
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Find the threshold that maximizes the Youden's Index (sum of sensitivity and specificity)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
y_cust_pred = (y_pred_proba > optimal_threshold).astype(int)


# make data with Ids and survival
X_test_1["Predict_val"] = y_cust_pred
X_test_1['ConsTest'] = y_test

numbCorPred, numbWrongPred = SumPredict(df = X_test_1, minFreq = 0, maxFreq = 1)
print(numbCorPred / (numbCorPred + numbWrongPred) * 100)

sumTest = SumPredictPerSamp(df = X_test_1, minFreq = 0, maxFreq = 1)

print(sumTest["True_Positive_Ident"].sum() / sumTest["Total_Positive_Truth"].sum() * 100)


print(sumTest['Correctly_identified'].sum() / sumTest['Total_SNVs'].sum() * 100)

#sumTest.to_csv("IntraSnv_results/ampseq_ConsTest_freq_1_0_predictSum_balanced_adjustedProb.csv")

cm = confusion_matrix(y_test, y_cust_pred)
# Extract the true negative, false positive, false negative, and true positive values
tn, fp, fn, tp = cm.ravel()

# Calculate class error rates
class_0_error = fp / (tn + fp)  # Class 0 (ConsTest = 0)
class_1_error = fn / (fn + tp)  # Class 1 (ConsTest = 1)

# Print the class error rates
print(f"Class 0 Error Rate: {class_0_error:.4f}")
print(f"Class 1 Error Rate: {class_1_error:.4f}")

86.46027601641178
75.5307855626327
86.46027601641178
Class 0 Error Rate: 0.1175
Class 1 Error Rate: 0.2447


In [20]:
print(sumTest["True_Positive_Ident"].sum())
print(sumTest["Total_Positive_Truth"].sum())
print(sumTest["False_Positive"].sum())
print(sumTest["False_Negative"].sum())

1423
1884
1354
461
